In [11]:
from multiprocessing import Pool
import numpy as np
from functools import reduce
from bs4 import BeautifulSoup
from selenium import webdriver
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import requests
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from difflib import SequenceMatcher
import requests
from unidecode import unidecode

In [12]:
request_data = {6812735111215: {6812735275055}, 6812735275055: {6812735111215}, 6813243408431: {6813230235695}, 8392243675465: {8392204976457, 8402364531017, 8401983996233, 8401874288969, 8392250261833}, 8402364531017: {8392243675465}, 8392204976457: {8392243675465}, 8401983996233: {8392243675465}, 8392250261833: {8392243675465}, 8401874288969: {8392243675465}, 8040171897126: set(), 8360770568521: {8209443914022, 4714200465455, 6971569405999, 4716683296815, 4292522377263}, 4422838091823: {8184214257958, 8205882851622, 8145297703206, 4716141838383, 4423408025647, 4716098453551}, 4423408025647: {4422838091823}, 8145297703206: {4422838091823}, 8184214257958: {4422838091823}, 8205882851622: {4422838091823}, 4292522377263: {8360770568521}, 6971569405999: {8360770568521}, 4714200465455: {8360770568521}, 4716683296815: {8360770568521}, 8209443914022: {8360770568521}, 4716141838383: {4422838091823}, 4716098453551: {4422838091823}, 8145438605606: {4484992991279, 4478129897519, 8138796269862, 8121597198630, 8183368352038, 8138802561318, 7110895632431, 7110849298479, 7111055769647, 4485367201839, 6750326423599, 6744294293551, 4485332926511, 4485857443887, 4484762599471, 4485271388207, 4485286297647, 4485033721903}, 6744294293551: {8121535627558, 4720178790447, 8145438605606, 6855695007791}, 8138796269862: {8145438605606}, 8138810360102: {8183404364070, 4484777771055, 8121535627558, 6855701463087}, 6750326423599: {8145438605606}, 7111055769647: {8145438605606}, 4485367201839: {8145438605606}, 4485332926511: {8145438605606}, 4485857443887: {8145438605606}, 8121535627558: {8138810360102, 6744294293551}, 6855695007791: {6744294293551}, 4484762599471: {8145438605606}, 6855701463087: {8138810360102}, 4485271388207: {8145438605606}, 4485286297647: {8145438605606}, 4485033721903: {8145438605606}, 4720178790447: {6744294293551}, 8121597198630: {8145438605606}, 8183368352038: {8145438605606}, 8183404364070: {8138810360102}, 8138802561318: {8145438605606}, 7110895632431: {8145438605606}, 7110849298479: {8145438605606}, 4484777771055: {8138810360102}, 4484992991279: {8145438605606}, 4478129897519: {8145438605606}, 6813230235695: {6813243408431}, 4394026303535: {6918198001711, 8046123548966, 8046111228198, 8046130790694, 8171997233446, 8171957420326, 8026522779942, 1809121837100, 4246802399276, 4253753901100, 7045826379823, 7045818056751, 7045824938031, 6831887810607, 6831885877295, 6831672262703, 6899726712879, 6996277919791, 6996321239087, 7090625380399, 6899760660527, 6772981923887, 8026510065958, 7989826224422, 8216110661926, 7090632917039, 7090669092911, 7090617647151, 7153376690223, 7090689998895, 6773024915503, 6899754795055, 6772980482095, 6899749716015, 6899757645871}, 8046130790694: {4394026303535}, 8046123548966: {4394026303535}, 8046111228198: {4394026303535}, 6831885877295: {4394026303535}, 6831887810607: {4394026303535}, 6831672262703: {4394026303535}, 6899726712879: {4394026303535}, 6996277919791: {4394026303535}, 6996321239087: {4394026303535}, 6831643721775: {7989719925030, 8169006301478, 8002512486694, 7989735883046, 6899827900463, 6831645687855, 6918250004527, 6918231162927, 6918229164079, 6899833503791, 6899825573935, 6899812335663, 6899823214639}, 7090625380399: {4394026303535}, 7090632917039: {4394026303535}, 7090669092911: {4394026303535}, 7090617647151: {4394026303535}, 1809121837100: {4394026303535}, 8171997233446: {4394026303535}, 8171957420326: {4394026303535}, 7045818056751: {4394026303535}, 7045824938031: {4394026303535}, 7045826379823: {4394026303535}, 4246802399276: {4394026303535}, 7153376690223: {4394026303535}, 8026522779942: {4394026303535}, 4253753901100: {4394026303535}, 8026510065958: {4394026303535}, 6918250004527: {6831643721775}, 6831645687855: {6831643721775}, 7090689998895: {4394026303535}, 8169006301478: {6831643721775}, 7989826224422: {4394026303535}, 6773024915503: {4394026303535}, 6899754795055: {4394026303535}, 6772980482095: {4394026303535}, 6899749716015: {4394026303535}, 6899757645871: {4394026303535}, 6899760660527: {4394026303535}, 6772981923887: {4394026303535}, 8216110661926: {4394026303535}, 7989719925030: {6831643721775}, 6899827900463: {6831643721775}, 6918231162927: {6831643721775}, 8002512486694: {6831643721775}, 6918229164079: {6831643721775}, 7989735883046: {6831643721775}, 6899833503791: {6831643721775}, 6899825573935: {6831643721775}, 6899812335663: {6831643721775}, 6899823214639: {6831643721775}, 6918198001711: {4394026303535}}

In [13]:
request_data

{6812735111215: {6812735275055},
 6812735275055: {6812735111215},
 6813243408431: {6813230235695},
 8392243675465: {8392204976457,
  8392250261833,
  8401874288969,
  8401983996233,
  8402364531017},
 8402364531017: {8392243675465},
 8392204976457: {8392243675465},
 8401983996233: {8392243675465},
 8392250261833: {8392243675465},
 8401874288969: {8392243675465},
 8040171897126: set(),
 8360770568521: {4292522377263,
  4714200465455,
  4716683296815,
  6971569405999,
  8209443914022},
 4422838091823: {4423408025647,
  4716098453551,
  4716141838383,
  8145297703206,
  8184214257958,
  8205882851622},
 4423408025647: {4422838091823},
 8145297703206: {4422838091823},
 8184214257958: {4422838091823},
 8205882851622: {4422838091823},
 4292522377263: {8360770568521},
 6971569405999: {8360770568521},
 4714200465455: {8360770568521},
 4716683296815: {8360770568521},
 8209443914022: {8360770568521},
 4716141838383: {4422838091823},
 4716098453551: {4422838091823},
 8145438605606: {4478129897519

In [14]:
product_links = [{'product alternates': ['https://www.woolsboutiqueuomo.com/products/handamde-cashmere-scarf-19andreas47', 'https://www.woolsboutiqueuomo.com/products/cashmere-shrug-stole-shawl-multicolor']}, {'product alternates': ['https://www.woolsboutiqueuomo.com/products/damask-medallions-cashmere-silk-scarf', 'https://www.woolsboutiqueuomo.com/products/amelie-flowers-cashmere-scarf']}, {'product alternates': ['https://www.woolsboutiqueuomo.com/products/borriello-light-blue-classic-denim-shirt', 'https://www.woolsboutiqueuomo.com/products/borriello-dark-blue-classic-denim-shirt', 'https://www.woolsboutiqueuomo.com/products/borriello-light-blue-shirt-poplin-cotton', 'https://www.woolsboutiqueuomo.com/products/borriello-poplin-cotton-white-shirt', 'https://www.woolsboutiqueuomo.com/products/borriello-light-blue-white-striped-shirt', 'https://www.woolsboutiqueuomo.com/products/borriello-light-blue-western-denim-shirt']}, {'product alternates': ['https://www.woolsboutiqueuomo.com/products/caramel-palm-cashmere-scarf-19andreas47']}, {'product alternates': ['https://www.woolsboutiqueuomo.com/products/smoke-grey-suit-merino-wool-loro-piana', 'https://www.woolsboutiqueuomo.com/products/caruso-brownish-green-wool-solaro-suit', 'https://www.woolsboutiqueuomo.com/products/caruso-wool-suit-autumn-winter', 'https://www.woolsboutiqueuomo.com/products/wool-solaro-suit-caruso-menswear', 'https://www.woolsboutiqueuomo.com/products/caruso-menswear-suit-spring-summer-blu', 'https://www.woolsboutiqueuomo.com/products/caruso-menswear-smoking-tuxedo-dark-blue']}, {'product alternates': ['https://www.woolsboutiqueuomo.com/products/loden-green-linen-wool-jacket-caruso', 'https://www.woolsboutiqueuomo.com/products/brown-marling-evans-wool-glen-check-blazer', 'https://www.woolsboutiqueuomo.com/products/indigo-blue-linen-jacket-caruso', 'https://www.woolsboutiqueuomo.com/products/caruso-wool-jacket-double-breasted-blue', 'https://www.woolsboutiqueuomo.com/products/caruso-doublebreasted-blazer-spring', 'https://www.woolsboutiqueuomo.com/products/caruso-mens-wool-jacket-springsummer', 'https://www.woolsboutiqueuomo.com/products/caruso-menswear-wool-jacket-blu']}, {'product alternates': ['https://www.woolsboutiqueuomo.com/products/brown-wool-crewneck-sweater-cruciani', 'https://www.woolsboutiqueuomo.com/products/pervinca-blue-cotton-sweater-cruciani', 'https://www.woolsboutiqueuomo.com/products/army-green-cotton-crewneck-sweater', 'https://www.woolsboutiqueuomo.com/products/green-cotton-polo-shirt-cruciani', 'https://www.woolsboutiqueuomo.com/products/steel-blue-merino-wool-crewneck-sweater', 'https://www.woolsboutiqueuomo.com/products/tshirt-ossigeno-cotton-army-green-cruciani', 'https://www.woolsboutiqueuomo.com/products/cruciani-linen-sailor-tshirt-marinara', 'https://www.woolsboutiqueuomo.com/products/mens-white-polo-long-sleeve-cruciani', 'https://www.woolsboutiqueuomo.com/products/cruciani-mens-wool-vests-blu', 'https://www.woolsboutiqueuomo.com/products/cruciani-mens-wool-sweater-crewneck-raglan-sleeve-blu', 'https://www.woolsboutiqueuomo.com/products/cruciani-mens-vneck-sweater-wool-blu', 'https://www.woolsboutiqueuomo.com/products/turtleneck-wool-jumper-dark-blue-cruciani', 'https://www.woolsboutiqueuomo.com/products/turtleneck-wool-jumper-smoke-grey-cruciani', 'https://www.woolsboutiqueuomo.com/products/army-green-crewneck-jumper-cashmere-silk-cruciani', 'https://www.woolsboutiqueuomo.com/products/cruciani-sweater-cashmere-silk-grey', 'https://www.woolsboutiqueuomo.com/products/cruciani-wool-cardigan-blue', 'https://www.woolsboutiqueuomo.com/products/cruciani-mens-cashmere-sweater-blue', 'https://www.woolsboutiqueuomo.com/products/cruciani-mens-tshirt-ossigeno-cotton-long-sleeve-white', 'https://www.woolsboutiqueuomo.com/products/cruciani-tshirt-ossigeno-cotton-white', 'https://www.woolsboutiqueuomo.com/products/cruciani-mens-tshirt-ossigeno-cotton-blue', 'https://www.woolsboutiqueuomo.com/products/cruciani-crewneck-cashmere-silk-marino', 'https://www.woolsboutiqueuomo.com/products/cruciani-sweater-cashmere-silk-nightblue', 'https://www.woolsboutiqueuomo.com/products/cruciani-mens-tshirt-ossigeno-cotton-long-sleeve-blu', 'https://www.woolsboutiqueuomo.com/products/cruciani-vneck-sweater-cashmere-silk-grey', 'https://www.woolsboutiqueuomo.com/products/cruciani-sweater-cashmere-silk-marino-blue', 'https://www.woolsboutiqueuomo.com/products/cruciani-vneck-sweater-cashmere-silk-blue']}, {'product alternates': ['https://www.woolsboutiqueuomo.com/products/fmarino-grenadine-silk-tie-regimental-ruby', 'https://www.woolsboutiqueuomo.com/products/barleycorn-tweed-tie-rust-brown', 'https://www.woolsboutiqueuomo.com/products/barleycorn-tweed-tie-apple-green', 'https://www.woolsboutiqueuomo.com/products/barleycorn-tweed-tie-denim-blue', 'https://www.woolsboutiqueuomo.com/products/crepe-silk-wool-tie-paisley-brandy-beige', 'https://www.woolsboutiqueuomo.com/products/crepe-silk-wool-tie-paisley-indigo-blue', 'https://www.woolsboutiqueuomo.com/products/flannel-wool-tie-princeofwales-lime-green', 'https://www.woolsboutiqueuomo.com/products/fmarino-silktie-octagon-royalblue', 'https://www.woolsboutiqueuomo.com/products/fmarino-wool-tie-chocolate-brown', 'https://www.woolsboutiqueuomo.com/products/fmarino-wooltie-princeofwales-turquoise', 'https://www.woolsboutiqueuomo.com/products/brown-silk-tie-chunky-gauze-fmarino', 'https://www.woolsboutiqueuomo.com/products/f-marino-handmade-3-fold-untipped-diamond-silk-tie-cream', 'https://www.woolsboutiqueuomo.com/products/light-blue-sky-flamed-wool-tie-drapes', 'https://www.woolsboutiqueuomo.com/products/light-flamed-wool-tie-drapes-burgundy', 'https://www.woolsboutiqueuomo.com/products/forest-green-flamed-wool-tie-drapes', 'https://www.woolsboutiqueuomo.com/products/denim-blue-big-paisley-silk-tie-fmarino', 'https://www.woolsboutiqueuomo.com/products/curry-paisley-print-silk-tie-fmarino', 'https://www.woolsboutiqueuomo.com/products/fmarino-bohemian-silk-tie-sky', 'https://www.woolsboutiqueuomo.com/products/bohemien-paisley-print-silk-tie-rust', 'https://www.woolsboutiqueuomo.com/products/checked-wool-tie-gold-magenta-fmarino', 'https://www.woolsboutiqueuomo.com/products/fmarino-silk-tie-regimental-ruby', 'https://www.woolsboutiqueuomo.com/products/grenadine-silk-tie-regimental-stripes-silver', 'https://www.woolsboutiqueuomo.com/products/navy-blue-flannel-wool-tie-windowpane', 'https://www.woolsboutiqueuomo.com/products/fmarino-grenadine-silk-tie-regimental-blue', 'https://www.woolsboutiqueuomo.com/products/seven-folds-grenadine-silk-tie-cobalt-blue', 'https://www.woolsboutiqueuomo.com/products/fmarino-grenadine-silk-tie-stripes-navy', 'https://www.woolsboutiqueuomo.com/products/fmarino-checked-wool-tie-rust-brown', 'https://www.woolsboutiqueuomo.com/products/cyan-blue-checks-flannel-wool-tie', 'https://www.woolsboutiqueuomo.com/products/fmarino-grenadine-silk-tie-regimental-stone', 'https://www.woolsboutiqueuomo.com/products/fmarino-silktie-regimental-teal-green', 'https://www.woolsboutiqueuomo.com/products/cherry-red-checks-tweed-tie-wools-boutique-uomo', 'https://www.woolsboutiqueuomo.com/products/countryside-checks-tweed-tie-wools-boutique-uomo', 'https://www.woolsboutiqueuomo.com/products/grey-checks-wool-tie-wools-boutique-uomo', 'https://www.woolsboutiqueuomo.com/products/mud-flannel-wool-tie-fmarino-napoli', 'https://www.woolsboutiqueuomo.com/products/mud-green-glen-check-tweed-tie-wools-boutique-uomo', 'https://www.woolsboutiqueuomo.com/products/fmarino-regimental-silk-tie-magenta']}, {'product alternates': ['https://www.woolsboutiqueuomo.com/products/grenadine-silk-tie-blue-wools-boutique-uomo', 'https://www.woolsboutiqueuomo.com/products/canary-yellow-grenadine-silk-tie-gauze', 'https://www.woolsboutiqueuomo.com/products/grenadine-silk-tie-chunky-gauze-copper-brown', 'https://www.woolsboutiqueuomo.com/products/brick-red-grenadine-silk-tie-chunky-gauze', 'https://www.woolsboutiqueuomo.com/products/fmarino-checked-wool-tie-hollandsherry-blue', 'https://www.woolsboutiqueuomo.com/products/fmarino-silk-tie-chunky-gauze-bronze', 'https://www.woolsboutiqueuomo.com/products/glen-check-tie-wool-curry-fmarino-napoli', 'https://www.woolsboutiqueuomo.com/products/regimental-gauze-silk-tie-denim-blue', 'https://www.woolsboutiqueuomo.com/products/burgundy-grenadine-silk-tie-chunky-gauze', 'https://www.woolsboutiqueuomo.com/products/cobalt-blue-silk-tie-chunky-gauze-fmarino', 'https://www.woolsboutiqueuomo.com/products/fmarino-silk-tie-chunky-gauze-dark-blue', 'https://www.woolsboutiqueuomo.com/products/fmarino-silk-tie-chunky-gauze-grass-green', 'https://www.woolsboutiqueuomo.com/products/fmarino-silk-tie-chunky-gauze-mud-brown', 'https://www.woolsboutiqueuomo.com/products/fmarino-silk-tie-chunky-gauze-pine-green']}]